In [2]:
# A-1 获取access_token等参数信息
# encoding:utf-8
# client_id 为官网获取的AK， client_secret 为官网获取的SK
import requests 
import pandas as pd
import json

host = 'https://aip.baidubce.com/oauth/2.0/token?'
params = {
    "grant_type":"client_credentials", # grant_type=client_credentials&client_id
    "client_id":"sppC66DFagDjKGKkrz6Q6X91", # 【官网获取的AK】
    "client_secret":"sTCBeBNyVY5HEPoAa7bywbdxaPI7nPcp" # 【官网获取的SK】
}
response = requests.get(host,params=params)
display(pd.json_normalize(response.json()).T) # T改变为上下排列

access_token=response.json()["access_token"]
access_token

,0
refresh_token,25.4d4f99b53ea62461cec596a5e4853a97.315360000....
expires_in,2592000
session_key,9mzdDoV6ey2oSQ+bbd0vcOrqnxmoWxyOohwC80pTv8ODbR...
access_token,24.57421ab138b2eefab7f8b9505930a0c6.2592000.16...
scope,public nlp_simnet nlp_wordemb nlp_comtag nlp_d...
session_secret,21010eeb23e73318a7c75cabc169e63c


'24.57421ab138b2eefab7f8b9505930a0c6.2592000.1609140397.282335-23019635'

## 应用技术
### 文章标签

In [10]:
def baidu_tag(title,content):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/keyword?access_token="+access_token
    data = {
        "title" : title,
        "content" : content
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [30]:
result01 = baidu_tag(title="如何评价红米发布 Redmi Note 9 4G，4G 市场还有多少需求？",content="我身边就有很多购机预算只有1000块左右的人，比如我爸妈、爷爷奶奶，还有亲戚家的老人。他们对手机的需求非常简单，聊微信、刷抖音，我爸有时候开车用下地图导航，我妈有空的时候则喜欢用手机K歌，当然了，是他们青春年代的歌。对他们来说，手机只要能流畅运行这几个常用APP就够了，音量也要大，什么5G、高刷对他们来说不仅徒增成本，还徒增功耗，他们宁愿手机续航时间更长一点。对了，他们选购手机时还有一点非常重要：手机要耐摔。虽然只要千把块钱，但万一手机摔坏了，他们会非常非常心疼、懊恼，甚至比我们摔了5、6000块的手机还心疼。所以我给他们选手机时，一定会特别注意这款手机屏幕有没有用康宁大猩猩玻璃，好让他们可以安心使用。")
result01gbk = str(result01,encoding="GBK")
result01dict =json.loads(result01gbk)
result01dict

{'log_id': 1103567917638322972,
 'items': [{'score': 0.935147, 'tag': '手机'}, {'score': 0.799591, 'tag': '数码'}]}

### 文章分类

In [14]:
def baidu_classify(title,content):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/topic?access_token="+access_token
    data = {
        "title" : title,
        "content" : content
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [29]:
result02 = baidu_classify(title="欧洲冠军联赛",content="欧洲冠军联赛是欧洲足球协会联盟主办的年度足球比赛，代表欧洲俱乐部足球最高荣誉和水平，被认为是全世界最高素质、最具影响力以及最高水平的俱乐部赛事，亦是世界上奖金最高的足球赛事和体育赛事之一。")
result02gbk = str(result02,encoding="GBK")
result02dict =json.loads(result02gbk)
result02dict
# display(pd.json_normalize(result02dict['item']))

{'log_id': 4049952978508071388,
 'item': {'lv2_tag_list': [{'score': 0.877436, 'tag': '足球'},
   {'score': 0.793682, 'tag': '国际足球'},
   {'score': 0.775911, 'tag': '英超'}],
  'lv1_tag_list': [{'score': 0.824329, 'tag': '体育'}]}}

### 文本纠错

In [31]:
def baidu_amend(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/ecnet?access_token="+access_token
    data = {
        "text":text
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [36]:
result03 = baidu_amend(text="明明明白白白的原因，但明明就想让白白把事情说的明明白白")
result03gbk = str(result03,encoding="GBK")
result03dict =json.loads(result03gbk)
result03dict

{'log_id': 2247225581012071484,
 'item': {'vec_fragment': [{'ori_frag': '的',
    'begin_pos': 44,
    'correct_frag': '得',
    'end_pos': 46}],
  'score': 2.20157,
  'correct_query': '明明明白白白的原因，但明明就想让白白把事情说得明明白白'},
 'text': '明明明白白白的原因，但明明就想让白白把事情说的明明白白'}

### 新闻摘要

In [37]:
def baidu_new(content,max_summary_len,title=""):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/news_summary?access_token="+access_token
    data = {
        "content":content,
        "max_summary_len":max_summary_len,
        "title":title
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [38]:
result04 = baidu_new(content="新华社三亚11月28日电 “奋斗者”号全海深载人潜水器成功完成万米海试并于28日胜利返航。中共中央总书记、国家主席、中央军委主席习近平发来贺信，致以热烈的祝贺，向所有致力于深海装备研发、深渊科学研究的科研工作者致以诚挚的问候。习近平在贺信中指出，“奋斗者”号研制及海试的成功，标志着我国具有了进入世界海洋最深处开展科学探索和研究的能力，体现了我国在海洋高技术领域的综合实力。从“蛟龙”号、“深海勇士”号到今天的“奋斗者”号，你们以严谨科学的态度和自立自强的勇气，践行“严谨求实、团结协作、拼搏奉献、勇攀高峰”的中国载人深潜精神，为科技创新树立了典范。习近平希望所有致力于深海装备研发、深渊科学研究的科研工作者继续弘扬科学精神，勇攀深海科技高峰，为加快建设海洋强国、为实现中华民族伟大复兴的中国梦而努力奋斗，为人类认识、保护、开发海洋不断作出新的更大贡献。在28日举行的“奋斗者”号海试返航欢迎活动上，中共中央政治局委员、国务院副总理刘鹤以视频连线形式宣读了习近平的贺信并讲话。他表示，习近平总书记的贺信充分体现了党中央对科技创新和海洋强国建设的高度重视，为做好深海科技工作指明了方向、提出了要求，要深入学习领会，坚决贯彻落实。“奋斗者”号在研制过程中充分调动和统筹各方科研力量，成功突破多项关键技术，是我国深海科技探索道路上的重要里程碑。要认真总结成功经验，大力弘扬中国载人深潜精神，强化产学研协同创新，加快科研成果应用转化，推动更多核心技术突破和可持续迭代。",max_summary_len="200",title="习近平致信祝贺“奋斗者”号全海深载人潜水器成功完成万米海试并胜利返航")
result04gbk = str(result04,encoding="GBK")
result04dict =json.loads(result04gbk)
result04dict

{'log_id': 2335668722297778524,
 'summary': '新华社三亚11月28日电 “奋斗者”号全海深载人潜水器成功完成万米海试并于28日胜利返航。习近平在贺信中指出，“奋斗者”号研制及海试的成功，标志着我国具有了进入世界海洋最深处开展科学探索和研究的能力，体现了我国在海洋高技术领域的综合实力。在28日举行的“奋斗者”号海试返航欢迎活动上，中共中央政治局委员、国务院副总理刘鹤以视频连线形式宣读了习近平的贺信并讲话。“奋斗者”号在研制过程中充分调动和统筹各方科研力量，成功突破多项关键技术，是我国深海科技探索道路上的重要里程碑。'}

### 评论观点抽取

In [44]:
def baidu_comment(text,type=""):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag?access_token="+access_token
    data = {
        "text":text,
        "type":type
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [48]:
result05 = baidu_comment(text="每次来迪拜都会住的酒店，每次都能呆一周以上因为实在是太舒服了好吗、管家很贴心还记得我加了what’s app 之后基本上每天出行定饭店还有各种需求都立马满足，而且这房间睡的也太舒服了，楼下的大客厅有沙发床很大足够一个大人睡了，楼下的大沙发也超级舒适真的超爽. 早餐的变化不多但是选择很多我对早餐要求很低因为房间住的太舒服了每天都睡到自然醒，按摩浴缸超大超舒适。 我们住在22层能看到整个Palm 和海滩，酒店室内泳池超级美、室外泳池这次有些冷没打卡但之前去过也很适合拍照。楼底的海底餐厅很棒是我心头最爱，这里的餐厅包括大堂的门童喝服务人员对酒店客人的服务态度特别好，因为偶经常有游客来参观但因为酒店管理人员出入很严格没有餐厅预订不会随意放人进来的所以人流量不会像亚特兰蒂斯那样，也没有乱哄哄的感觉，总之你是住店客人的话都是不一样的待遇。基本上每年都会来打卡明年期待再见",type="1")
result05gbk = str(result05,encoding="GBK")
result05dict =json.loads(result05gbk)
# result05dict
d = (pd.json_normalize(result05dict["items"]).T)
d

,0,1,2,3,4,5,6,7,8
sentiment,2,2,2,2,2,2,2,2,2
abstract,每次都能呆一周以上因为实在是太舒服了好吗<span></span>,管家很贴心还记得我加了what’s<span></span>,楼下的大沙发也超级舒适真的超爽.<span></span>,<span>早餐的变化不多但是选择很多我对早餐要求很低因为房间住的太舒服了每天都睡到自然醒<...,按摩浴缸超大超舒适<span></span>,<span>酒店室内泳池超级美</span>,<span>楼底的海底餐厅很棒是我心头最爱</span>,这里的餐厅包括大堂的门童喝服务人员对酒店客人的<span>服务态度特别好</span>,这里的餐厅包括大堂的门童喝服务人员对酒店客人的服务<span>态度特别好</span>
prop,感觉,管家,店内设施,房间不错,浴缸,酒店不错,餐厅不错,服务态度,态度
begin_pos,40,29,31,0,18,0,0,46,50
end_pos,40,29,31,78,18,18,30,60,60
adj,实在,贴心,完善,,舒适,,,好,好


### 情感倾向分析

In [49]:
def baidu_sentiment(text):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?access_token="+access_token
    data = {
        "text":text
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [52]:
result06 = baidu_sentiment(text="得得得得得得的得得得得得得")
result06gbk = str(result06,encoding="GBK")
result06dict =json.loads(result06gbk)
result06dict

{'log_id': 2121871665874984636,
 'text': '得得得得得得的得得得得得得',
 'items': [{'positive_prob': 0.979949,
   'confidence': 0.955443,
   'negative_prob': 0.0200508,
   'sentiment': 2}]}

### 对话情绪识别
#### 接口描述
可自动识别出当前会话者所表现出的一级和二级细分情绪类别及其置信度，针对正面和负面的情绪，还可给出参考回复话术

In [56]:
# 参数scene
#default（默认项-不区分场景）
#talk（闲聊对话-如度秘聊天等）
#task（任务型对话-如导航对话等）
#customer_service（客服对话-如电信/银行客服等）

def baidu_emotion(text,scene=""):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/emotion?access_token="+access_token
    data = {
        "text":text,
        "scene":scene
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [69]:
result07 = baidu_emotion(text="真不错",scene="default")
result07gbk = str(result07,encoding="GBK")
result07dict =json.loads(result07gbk)
result07dict

{'log_id': 6234573762542262044,
 'text': '真不错',
 'items': [{'subitems': [{'prob': 0.997125, 'label': 'like'}],
   'replies': ['感觉自己棒棒哒'],
   'prob': 0.997125,
   'label': 'optimistic'},
  {'subitems': [], 'replies': [], 'prob': 0.00155242, 'label': 'neutral'},
  {'subitems': [], 'replies': [], 'prob': 0.00132238, 'label': 'pessimistic'}]}

### 多实体情感倾向分析接口（可定制）
（邀测期，需发工单申请后方可使用，工单内容请填写：企业名称、APPID、业务需求，客服会第一时间回复并为您开通试用权限）
#### 接口描述
针对每一个系统识别到的主体，做自动情感倾向性判断（积极、消极、中性），并给出相应的置信度

### 地址识别
#### 接口描述
帮助精准提取快递填单文本中的姓名、电话、地址信息，通过自然语言处理辅助地址识别做自动补充和纠正，生成标准规范的结构化信息

In [70]:
def baidu_address(text,confidence=""):
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/address?access_token="+access_token
    data = {
        "text":text,
        "confidence":confidence
    }
    # unicode 国际编码识别中文
    encode_data=json.dumps(data).encode('UTF-8')
    headers = {
        'Content-Type':'application/json'
    }
    return requests.post(url,encode_data).content

In [79]:

result08 = baidu_address(text="广州市从化区中山大学南方学院文学与传媒学院新媒体研究中心 小麦",confidence=100)
result08dict = json.loads(result08)
result08dict

{'log_id': 1332609879726096384,
 'town': '温泉镇',
 'city': '广州市',
 'county_code': '440117',
 'county': '从化区',
 'city_code': '440100',
 'phonenum': '',
 'province_code': '440000',
 'town_code': '440117100',
 'province': '广东省',
 'person': '',
 'detail': '中山大学南方学院文学与传媒学院新媒体研究中心小麦',
 'text': '广州市从化区中山大学南方学院文学与传媒学院新媒体研究中心 小麦'}

### 消费者评论分析
（邀测期，需发工单申请后方可使用，工单内容请填写：企业名称、APPID、业务需求，客服会第一时间回复并为您开通试用权限）
#### 接口描述
准确提取评论中的核心观点，并支持自定义评论分类，判断消费者的情感极性（好评/差评），从评论的语言描述中，帮助企业更好理解消费者的想法和感受